<a href="https://colab.research.google.com/github/celynhs/Bangkit-project/blob/main/Grocery%20Products%20Recommender%20System%20BeBi(Beli_Bijak).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Grocery Product Recommender System**


# Upload Datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


import modules

In [ ]:
import math
import scipy
import numpy as np
import pandas as pd
import seaborn as sns

read dataset and convert them to dataframe using pandas

In [ ]:
# Read product info dataset
df7 = pd.read_csv('/content/drive/MyDrive/New folder/BigBasket Products.csv')

In [ ]:
# Read transaction dataset 
df1 = pd.read_csv('/content/drive/MyDrive/New folder/products.csv')
df2 = pd.read_csv('/content/drive/MyDrive/New folder/aisles.csv')
df8= pd.read_csv('/content/drive/MyDrive/New folder/departments.csv')
df9 = pd.read_csv('/content/drive/MyDrive/New folder/order_products__train.csv')
df10 = pd.read_csv('/content/drive/MyDrive/New folder/orders.csv')

# Data Exploratory

## Get the shapes of all the dataframes

In [ ]:
# get shapes of transaction data
print(df1.shape) # data of products
print(df2.shape) # data of aisle
print(df8.shape) # data of departments
print(df9.shape) # data of orders and products
print(df10.shape) # data of customer orders

In [ ]:
# get shapes of product details 
print(df7.shape)

## Display the datasets

In [ ]:
dataset_bigmarket = df7.copy()
dataset_bigmarket

## count the null value from each column of each dataset

In [ ]:
null_value = dict()
for column in dataset_bigmarket:
  null_value[column] = dataset_bigmarket[column].isna().sum()
print(f"null value of bigmarket dataset: {null_value}\n")

## get the amount of each unique value in the ratings column

In [ ]:
dataset_bigmarket.rename(columns = {'rating':'ratings', 'sale_price':'discount_price', 'actual_price':'market_price'}, inplace = True)
vals = dict(dataset_bigmarket["ratings"].value_counts())
print(f"{vals} \n")

## get the datatypes of each column

In [ ]:
# product dataset
dataset_bigmarket.dtypes

# Dataset preprocessing

## Bigmarket preprocessing

In [ ]:
# change type of description column
dataset_bigmarket['description'] = dataset_bigmarket['description'].astype(str)

In [ ]:
# drop null value (product without name / brand) and get train data
train_bm = dataset_bigmarket.dropna()
train_bm.isna().sum()

In [ ]:
# check shape
train_bm.shape

In [ ]:
train_bm.dtypes

## Transform & Merge Datasets

In [ ]:
train_bm.rename(columns = {'product':'name', 'market_price':'actual_price'}, inplace = True)

<ipython-input-15-6630b6182842>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_bm.rename(columns = {'product':'name', 'market_price':'actual_price'}, inplace = True)


In [ ]:
train_bm.dtypes

index               int64
name               object
category           object
sub_category       object
brand              object
discount_price    float64
actual_price      float64
type               object
ratings           float64
description        object
dtype: object

In [ ]:
product_dataset = train_bm[['name','category','sub_category','brand','ratings','discount_price','actual_price','description']]

In [ ]:
product_dataset.isna().sum()

name              0
category          0
sub_category      0
brand             0
ratings           0
discount_price    0
actual_price      0
description       0
dtype: int64

In [ ]:
product_dataset = product_dataset.drop_duplicates() # drop duplicate data
product_dataset.duplicated().sum() # check if there are any duplicates data

0

In [ ]:
# Add 1 to the index values
product_dataset.reset_index(inplace=True)

product_dataset['product_id'] = product_dataset.index + 1

<ipython-input-20-4c47c2a3ed76>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_dataset['product_id'] = product_dataset.index + 1


In [ ]:
# get products dataset
products = product_dataset[['product_id','name','actual_price','discount_price','ratings','brand','category','description']]
# merge instacart data
df12 = pd.merge(df2, df1, how='inner', on='aisle_id')
df128 = pd.merge(df12, df8, how='inner', on='department_id')
df1289 = pd.merge(df9, df128, how='inner', on='product_id')
dataset_instacart = pd.merge(df10, df1289, how='inner', on='order_id')

In [ ]:
print(products['product_id'].min())
print(products['product_id'].max())

1
18737


In [ ]:
print(dataset_instacart['product_id'].min())
print(dataset_instacart['product_id'].max())

1
49688


In [ ]:
# Drop rows where the value of product id is greater than 1000 (for data efficiency)
products = products.drop(products[products['product_id'] > 1000].index)
transactions = dataset_instacart.drop(dataset_instacart[dataset_instacart['product_id'] > 1000].index)

In [ ]:
print(transactions['product_id'].min())
print(transactions['product_id'].max())

1
1000


In [ ]:
print(products['product_id'].min())
print(products['product_id'].max())

1
1000


In [ ]:
transactions.dtypes

order_id                    int64
user_id                     int64
eval_set                   object
order_number                int64
order_dow                   int64
order_hour_of_day           int64
days_since_prior_order    float64
product_id                  int64
add_to_cart_order           int64
reordered                   int64
aisle_id                    int64
aisle                      object
product_name               object
department_id               int64
department                 object
dtype: object

In [ ]:
# drop irrelevant columns on transaction dataset 
transactions = transactions.drop(['department_id','department','product_name','order_id','eval_set','order_number','order_dow','order_hour_of_day','days_since_prior_order','aisle_id','aisle'], axis=1)

In [ ]:
merged_data = pd.merge(transactions, products, how='inner', on='product_id')

In [ ]:
merged_data = merged_data.sample(frac=1)
merged_data.head()

,user_id,product_id,add_to_cart_order,reordered,name,actual_price,discount_price,ratings,brand,category,description
13671,34504,768,10,1,Lemon Chaat - Pickle in your pocket,50.0,50.0,3.4,Go Desi,"Foodgrains, Oil & Masala",Lemon Chaat is tiny bits of dried lemon peel m...
11890,50009,311,12,1,Solitaire Storage Transparent Pet Container Se...,969.0,409.0,4.2,Steelo,"Kitchen, Garden & Pets",These Steelo storage containers are made from ...
14012,52002,116,4,0,Ragi Flakes,225.0,225.0,4.5,Murginns,Snacks & Branded Foods,"Made of only Ragi (Nachni) and Jaggery, we gua..."
18780,11393,86,18,0,Fructis Serum - Long & Strong,230.0,230.0,4.8,Garnier,Beauty & Hygiene,Garnier Fruits Long & Strong Strengthening Ser...
3680,29396,260,1,1,Fruit N Fibre Muesli - Mixed Fruit,320.0,320.0,4.3,Bagrrys,Snacks & Branded Foods,A delightful muesli for a great start to your ...


In [ ]:
df = merged_data.copy()
df.shape

(21299, 11)

In [ ]:
# download the data to google drive
filename = 'final_dataset.csv'

df.to_csv('/content/drive/MyDrive/' + filename)

In [ ]:
products

,product_id,name,actual_price,discount_price,ratings,brand,category,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,220.0,220.00,4.1,Sri Sri Ayurveda,Beauty & Hygiene,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,180.0,180.00,2.3,Mastercook,"Kitchen, Garden & Pets","Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",250.0,119.00,3.4,Trm,Cleaning & Household,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,176.0,149.00,3.7,Nakoda,Cleaning & Household,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,162.0,162.00,4.4,Nivea,Beauty & Hygiene,Nivea Creme Soft Soap gives your skin the best...
...,...,...,...,...,...,...,...,...
995,996,Alkaline Battery AAA-2B,80.0,70.00,4.3,Panasonic,"Kitchen, Garden & Pets",1. It is highly advisable to use batteries of ...
996,997,Strong & Healthy Shampoo,118.0,114.46,4.2,Meera,Beauty & Hygiene,Meera Strong & Healthy Shampoo contains Kunkud...
997,998,Amalfi Deo For Men,499.0,474.05,4.5,Skinn by Titan,Beauty & Hygiene,This vibrant and fresh Amalfi Deo For Men is i...
998,999,Organic - Nippat,90.0,90.00,4.5,Phalada Pure & Sure,Snacks & Branded Foods,"All Natural, Certified Organic, No Trans Fat, ..."


In [ ]:
average_ratings = df.groupby('product_id')['ratings'].mean().reset_index()
average_ratings.columns = ['product_id', 'average_rating']

productset = pd.merge(products, average_ratings, on='product_id', how='left')
productset['image'] = "https://upload.wikimedia.org/wikipedia/commons/thumb/6/65/No-Image-Placeholder.svg/495px-No-Image-Placeholder.svg.png?20200912122019"
productset

,product_id,name,actual_price,discount_price,ratings,brand,category,description,average_rating,image
0,1,Garlic Oil - Vegetarian Capsule 500 mg,220.0,220.00,4.1,Sri Sri Ayurveda,Beauty & Hygiene,This Product contains Garlic Oil that is known...,4.1,https://upload.wikimedia.org/wikipedia/commons...
1,2,Water Bottle - Orange,180.0,180.00,2.3,Mastercook,"Kitchen, Garden & Pets","Each product is microwave safe (without lid), ...",2.3,https://upload.wikimedia.org/wikipedia/commons...
2,3,"Brass Angle Deep - Plain, No.2",250.0,119.00,3.4,Trm,Cleaning & Household,"A perfect gift for all occasions, be it your m...",3.4,https://upload.wikimedia.org/wikipedia/commons...
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,176.0,149.00,3.7,Nakoda,Cleaning & Household,Multipurpose container with an attractive desi...,3.7,https://upload.wikimedia.org/wikipedia/commons...
4,5,Creme Soft Soap - For Hands & Body,162.0,162.00,4.4,Nivea,Beauty & Hygiene,Nivea Creme Soft Soap gives your skin the best...,4.4,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...,...,...,...,...,...,...
995,996,Alkaline Battery AAA-2B,80.0,70.00,4.3,Panasonic,"Kitchen, Garden & Pets",1. It is highly advisable to use batteries of ...,4.3,https://upload.wikimedia.org/wikipedia/commons...
996,997,Strong & Healthy Shampoo,118.0,114.46,4.2,Meera,Beauty & Hygiene,Meera Strong & Healthy Shampoo contains Kunkud...,4.2,https://upload.wikimedia.org/wikipedia/commons...
997,998,Amalfi Deo For Men,499.0,474.05,4.5,Skinn by Titan,Beauty & Hygiene,This vibrant and fresh Amalfi Deo For Men is i...,NaN,https://upload.wikimedia.org/wikipedia/commons...
998,999,Organic - Nippat,90.0,90.00,4.5,Phalada Pure & Sure,Snacks & Branded Foods,"All Natural, Certified Organic, No Trans Fat, ...",4.5,https://upload.wikimedia.org/wikipedia/commons...


In [ ]:
productset['average_rating'] =productset['average_rating'].fillna(0)
productset.isna().sum()

product_id        0
name              0
actual_price      0
discount_price    0
ratings           0
brand             0
category          0
description       0
average_rating    0
image             0
dtype: int64

In [ ]:
filename = 'final_product.csv'
productset.to_csv('/content/drive/MyDrive/' + filename)

# ML Model Development (Data Training & Evaluate Model)

## import modules

In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler # rescale and encode categorical data
from sklearn.model_selection import train_test_split # split data
# module to make neural network model for content based filtering
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from sklearn.metrics import mean_squared_error, mean_absolute_error # evaluate model 
# module to create collaborative filtering recommender system
from sklearn.utils import shuffle
from random import choices

## Load Data

In [ ]:
# Load the clean dataset
df = pd.read_csv('/content/drive/MyDrive/final_dataset.csv')

df = df.drop(["Unnamed: 0","description"],axis=1)

In [ ]:
# Split the data into training and testing sets (if not already done)
# Preprocessed features and target variable

# Scale numerical features
dataset = df.copy()
scaler = MinMaxScaler()
dataset[["actual_price", "discount_price", "ratings", "add_to_cart_order"]] = scaler.fit_transform(dataset[["actual_price", "discount_price", "ratings", "add_to_cart_order"]])

# Convert 'category' and 'brand' column to numeric labels
dataset['category_id'] = LabelEncoder().fit_transform(dataset['category'])
dataset['brand_id'] = LabelEncoder().fit_transform(dataset['brand'])


# Split the dataset into train and test sets
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

features = ["category_id", "discount_price", "reordered", "add_to_cart_order"]
x = train[features]
target = train["ratings"]

## Collaborative filtering 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import plot_model


# Define the number of users and items
num_users = len(dataset['user_id'].unique())
num_items = len(dataset['product_id'].unique())

# Convert user and item IDs to sequential integers
user_mapping = {user_id: i for i, user_id in enumerate(dataset['user_id'].unique())}
item_mapping = {item_id: i for i, item_id in enumerate(dataset['product_id'].unique())}

train_data  = train.copy()
test_data  = test.copy()
train_data['user'] = train_data['user_id'].map(user_mapping)
train_data['item'] = train_data['product_id'].map(item_mapping)
test_data['user'] = test_data['user_id'].map(user_mapping)
test_data['item'] = test_data['product_id'].map(item_mapping)

# Define the embedding dimensions and MLP layers
embedding_dim = 16
mlp_layers = [64, 32, 16]

# Define the input layers
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

# User and item embedding layers
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim)(user_input)
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim)(item_input)

# Flatten the embedding layers
user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)

# Concatenate the user and item embeddings
concat = Concatenate()([user_flatten, item_flatten])

# MLP layers
mlp_output = concat
for layer_size in mlp_layers:
    mlp_output = Dense(layer_size, activation='relu')(mlp_output)

# Output layer
output = Dense(1)(mlp_output)

# Create the NCF model
clfmodel = Model(inputs=[user_input, item_input], outputs=output)

# Compile the model
clfmodel.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model
clfmodel.fit([train_data['user'], train_data['item']], train_data['ratings'],
          epochs=10, batch_size=256, validation_data=([test_data['user'], test_data['item']], test_data['ratings']))

# Generate predictions on the test set
predictions = clfmodel.predict([test_data['user'], test_data['item']]).flatten()

# Evaluate the model using RMSE
mse = np.mean((predictions - test_data['ratings'])**2)
rmse = np.sqrt(mse)
print("RMSE: {:.4f}".format(rmse))


Epoch 1/10
67/67 [==============================] - 5s 50ms/step - loss: 0.2910 - val_loss: 0.0165
Epoch 2/10
67/67 [==============================] - 2s 25ms/step - loss: 0.0059 - val_loss: 0.0025
Epoch 3/10
67/67 [==============================] - 1s 11ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 4/10
67/67 [==============================] - 1s 8ms/step - loss: 3.7663e-04 - val_loss: 8.2739e-04
Epoch 5/10
67/67 [==============================] - 1s 9ms/step - loss: 1.6303e-04 - val_loss: 7.1273e-04
Epoch 6/10
67/67 [==============================] - 1s 17ms/step - loss: 8.1742e-05 - val_loss: 6.6943e-04
Epoch 7/10
67/67 [==============================] - 0s 7ms/step - loss: 4.5602e-05 - val_loss: 6.5056e-04
Epoch 8/10
67/67 [==============================] - 0s 5ms/step - loss: 2.7147e-05 - val_loss: 6.4183e-04
Epoch 9/10
67/67 [==============================] - 0s 5ms/step - loss: 1.7229e-05 - val_loss: 6.3925e-04
Epoch 10/10
134/134 [==============================] - 0s 2ms/step

In [ ]:
user_id = 126093  # Replace with the ID of the desired user
num_recommendations = 10  # Number of recommendations to generate

# Generate item recommendations for the specific user
user_items = np.full((num_items,), user_mapping[user_id])
item_ids = np.arange(num_items)

# Predict ratings for the user-item combinations
predictions = clfmodel.predict([user_items, item_ids]).flatten()

# Create a DataFrame of predictions for the user
recommendations_df = pd.DataFrame({'product_id': item_mapping.keys(), 'rating': predictions})

# Sort the DataFrame by predicted rating in descending order
recommendations_df = recommendations_df.sort_values('rating', ascending=False)

# Select the top 10 recommendations
top_10_recs = recommendations_df.nlargest(num_recommendations, 'rating')

# Display the top 10 recommendations for the user
print(top_10_recs[['product_id', 'rating']])

25/25 [==============================] - 0s 2ms/step
     product_id    rating
332         925  1.001761
145          94  1.000964
407         977  1.000534
62           45  0.999853
85          920  0.999554
416         669  0.999304
43          169  0.999218
80          907  0.999007
93           25  0.998842
429         734  0.998589


## Content based filtering

In [ ]:
# Split the data into training and testing sets (if not already done)
# Preprocessed features and target variable

# Scale numerical features
dataset = df.copy()
scaler = MinMaxScaler()
dataset[["actual_price", "discount_price", "ratings", "add_to_cart_order"]] = scaler.fit_transform(dataset[["actual_price", "discount_price", "ratings", "add_to_cart_order"]])

# Convert 'category' and 'brand' column to numeric labels
dataset['category_id'] = LabelEncoder().fit_transform(dataset['category'])
dataset['brand_id'] = LabelEncoder().fit_transform(dataset['brand'])


# Split the dataset into train and test sets
train, test = train_test_split(dataset, test_size=0.2, random_state=42)

features = ["category_id", "discount_price", "reordered", "add_to_cart_order"]
x = train[features]
target = train["ratings"]

### Neural network

In [ ]:
# Define input layers
#brand_input = Input(shape=(1,))
category_input = Input(shape=(1,))
other_features_input = Input(shape=(3,))

# Embedding layers for user ID and category ID
#brand_embedding = Embedding(input_dim=473, output_dim=32)(brand_input)
category_embedding = Embedding(input_dim=9, output_dim=32)(category_input)

# Flatten the embeddings
#user_flatten = Flatten()(user_embedding)
category_flatten = Flatten()(category_embedding)

# Concatenate the embeddings and other features
concatenated = Concatenate()([category_flatten, other_features_input])

# Dense layers
x = Dense(64, activation='relu')(concatenated)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# Output layer
output = Dense(1, activation='linear')(x)

# Create the model
cbfmodel = Model(inputs=[category_input, other_features_input], outputs=output)

# Compile and train the model
cbfmodel.compile(optimizer='adam', loss='mean_squared_error')
cbfmodel.fit([train['category_id'], train[["discount_price", "reordered", "add_to_cart_order"]]], target, epochs=10,batch_size=32)



Epoch 1/10
533/533 [==============================] - 4s 5ms/step - loss: 0.0383
Epoch 2/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0289
Epoch 3/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0283
Epoch 4/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0269
Epoch 5/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0256
Epoch 6/10
533/533 [==============================] - 2s 4ms/step - loss: 0.0249
Epoch 7/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0243
Epoch 8/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0240
Epoch 9/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0237
Epoch 10/10
533/533 [==============================] - 2s 3ms/step - loss: 0.0235


**GET PREDICTION**

In [ ]:
# Specify the user ID for which you want to make recommendations
target_user_id = 126093
# Predict the ratings for all products in the dataset
all_products_data = dataset[dataset['user_id'] != target_user_id]
X_all = [all_products_data['category_id'], all_products_data[["discount_price", "reordered", "add_to_cart_order"]]]
predicted_ratings = cbfmodel.predict(X_all).flatten()


# Create a list of tuples containing product ID and predicted rating
product_ratings = list(zip(all_products_data['product_id'], predicted_ratings))

# Sort the product ratings by predicted rating in descending order
product_ratings.sort(key=lambda x: x[1], reverse=True)

# Get the top 10 recommendations without duplicate product IDs
recommended_products = []
recommended_product_ids = set()

for product_id, _ in product_ratings:
    if product_id not in recommended_product_ids:
        recommended_products.append((product_id, _))
        recommended_product_ids.add(product_id)
    if len(recommended_products) >= 10:
        break

# Filter the original dataset based on the recommended product IDs
top_10_recommendations = all_products_data[
    all_products_data['product_id'].isin([p[0] for p in recommended_products])
]

# Remove duplicate product IDs from the recommendations
top_10_recommendations = top_10_recommendations.drop_duplicates(subset='product_id')

# Keep the top 10 recommendations (if there are more than 10 after removing duplicates)
top_10_recommendations = top_10_recommendations.head(10)
# Print the top 10 product recommendations
print(top_10_recommendations[['product_id', 'name','discount_price','category']])

666/666 [==============================] - 1s 1ms/step
       product_id                             name  discount_price  \
42            276             Ultimate Choco Berry        0.260930   
83            778                  Wafer - Vanille        0.052464   
91             45               Plain Green Olives        0.046912   
143           920                 Hair Dryer HD180        0.551881   
968            18                   Salted Pumpkin        0.047189   
1910          635              Black Sliced Olives        0.047189   
2626          669                            Syrup        0.052741   
6710          698           Spice - Smoked Paprika        0.049702   
8927          813  White Radiance Whip - UV SPF 30        0.468883   
15564         395          For Man Eau De Toilette        0.667592   

                     category  
42     Snacks & Branded Foods  
83       Gourmet & World Food  
91       Gourmet & World Food  
143          Beauty & Hygiene  
968      Gourm

**TEST/EVALUATEE MODEL**

In [ ]:
# test dataset
X_test = [test['category_id'], test[["discount_price", "reordered", "add_to_cart_order"]]]
predictions = cbfmodel.predict(X_test)

134/134 [==============================] - 0s 1ms/step


In [ ]:
# Assuming you have already trained and obtained predictions from your model
y_true = test['ratings']  # True labels
y_pred = predictions  # Predicted labels

mse = mean_squared_error(y_true, y_pred)

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_true, y_pred)

# Print the evaluation metrics
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

Mean Squared Error (MSE): 0.024716720253243445
Mean Absolute Error (MAE): 0.11132258945227788


# Combine Recommender System Model (**GET PREDICTION**)

## get the info from collaborative filtering model

In [ ]:
clfrecs = top_10_recs[['product_id', 'rating']].copy()
clfrecs = clfrecs.rename(columns={'rating': 'clf_ratings'})
clfinfo = pd.merge(clfrecs,df,on='product_id')
clfinfo = clfinfo.drop_duplicates(subset='product_id', keep='first')
clfinfo = clfinfo.drop(['user_id','add_to_cart_order','reordered','actual_price','ratings','brand'],axis=1) 
# Identify the index of the column to move
column_index = clfinfo.columns.get_loc('clf_ratings')

# Create a list of column names in the desired order
new_columns = list(clfinfo.columns[:column_index]) + list(clfinfo.columns[column_index+1:]) + ['clf_ratings']

# Reorder the columns in the DataFrame
clfinfo = clfinfo[new_columns]

## get the info from content based filtering model

In [ ]:
pred_rating = pd.DataFrame(product_ratings)
pred_rating.columns =['product_id','cbf_ratings']
pred_rating = pred_rating.drop_duplicates(subset='product_id', keep='first')
cbfrecs = top_10_recommendations[['product_id', 'name','discount_price','category']]
cbfinfo = pd.merge(cbfrecs, pred_rating, on='product_id')
cbfinfo = cbfinfo.drop_duplicates(subset='product_id', keep='first')
cbfinfo['discount_price'] = cbfinfo['discount_price'] * (df['discount_price'].max()- df['discount_price'].min() ) + df['discount_price'].min()


## get predicted ratings from content based filtering on top 10 collaborative filtering recommended items 

In [ ]:
finalclf =  pd.merge(clfinfo, pred_rating, on='product_id', how='left')
finalclf

,product_id,name,discount_price,category,clf_ratings,cbf_ratings
0,925,Hair Gel Regular Hold,22.50,Beauty & Hygiene,1.001761,0.691832
1,94,SPF 15 Sunscreen Lotion,209.30,Beauty & Hygiene,1.000964,0.720035
2,977,Styling Shampoo For Men - Cooling & Style,399.00,Beauty & Hygiene,1.000534,0.744919
3,45,Plain Green Olives,179.00,Gourmet & World Food,0.999853,0.858148
4,920,Hair Dryer HD180,1998.15,Beauty & Hygiene,0.999554,0.962413
5,669,Syrup,200.00,Gourmet & World Food,0.999304,0.852507
6,169,Lavender & Vanilla Handmade Luxury Soap,275.00,Beauty & Hygiene,0.999218,0.735281
7,907,Sauce - Aglio Olio,290.00,Gourmet & World Food,0.999007,0.817034
8,25,Veggie Cutter,195.00,"Kitchen, Garden & Pets",0.998842,0.683080
9,734,Lavender Try Me Personal Care Gift Set,850.00,Beauty & Hygiene,0.998589,0.762915


## get predicted ratings from collaborative filtering on top 10 content based filtering recommended items 

In [ ]:
user_id = 126093  # Replace with the ID of the desired user
product_ids = [x for x in cbfinfo['product_id']]  # Replace with the desired list of product IDs

# Generate user-item combinations for the specific user and product list
user_ids = np.full(len(product_ids), user_mapping[user_id])
item_ids = np.array([item_mapping[product_id] for product_id in product_ids])

# Predict ratings for the user-item combinations
predictions = clfmodel.predict([user_ids, item_ids]).flatten()

# Create a DataFrame of predictions for the user and product list
user_product_ratings_df = pd.DataFrame({
                                        'product_id': product_ids,
                                        'clf_ratings': predictions})

# Display the predicted ratings for the user and product list
user_product_ratings_df

1/1 [==============================] - 0s 21ms/step


,product_id,clf_ratings
0,276,0.975257
1,778,0.921420
2,45,0.999853
3,920,0.999554
4,18,0.968970
5,635,0.703465
6,669,0.999304
7,698,0.921263
8,813,0.993963
9,395,0.630803


In [ ]:
finalcbf =  pd.merge(cbfinfo, user_product_ratings_df, on='product_id', how='left')
finalcbf

,product_id,name,discount_price,category,cbf_ratings,clf_ratings
0,276,Ultimate Choco Berry,950.00,Snacks & Branded Foods,0.957850,0.975257
1,778,Wafer - Vanille,199.00,Gourmet & World Food,0.852862,0.921420
2,45,Plain Green Olives,179.00,Gourmet & World Food,0.858148,0.999853
3,920,Hair Dryer HD180,1998.15,Beauty & Hygiene,0.962413,0.999554
4,18,Salted Pumpkin,180.00,Gourmet & World Food,0.857688,0.968970
5,635,Black Sliced Olives,180.00,Gourmet & World Food,0.858076,0.703465
6,669,Syrup,200.00,Gourmet & World Food,0.852507,0.999304
7,698,Spice - Smoked Paprika,189.05,Gourmet & World Food,0.855368,0.921263
8,813,White Radiance Whip - UV SPF 30,1699.15,Beauty & Hygiene,0.908941,0.993963
9,395,For Man Eau De Toilette,2415.00,Beauty & Hygiene,0.912085,0.630803


## Combine final dataset

In [ ]:
combined = pd.concat([finalclf, finalcbf], axis=0)
combined['cbf_ratings'] = combined['cbf_ratings'] * (df['ratings'].max() - df['ratings'].min() ) + df['ratings'].min()
combined['cbf_ratings'] = combined['cbf_ratings'].clip(lower=0.0, upper=5.0)
combined['clf_ratings'] = combined['clf_ratings'] * (df['ratings'].max() - df['ratings'].min() ) + df['ratings'].min()
combined['clf_ratings'] = combined['clf_ratings'].clip(lower=0.0, upper=5.0)
combined = combined.drop_duplicates(subset='product_id', keep='first')
combined

,product_id,name,discount_price,category,clf_ratings,cbf_ratings
0,925,Hair Gel Regular Hold,22.50,Beauty & Hygiene,5.000000,3.767328
1,94,SPF 15 Sunscreen Lotion,209.30,Beauty & Hygiene,5.000000,3.880140
2,977,Styling Shampoo For Men - Cooling & Style,399.00,Beauty & Hygiene,5.000000,3.979675
3,45,Plain Green Olives,179.00,Gourmet & World Food,4.999412,4.432593
4,920,Hair Dryer HD180,1998.15,Beauty & Hygiene,4.998215,4.849652
5,669,Syrup,200.00,Gourmet & World Food,4.997217,4.410027
6,169,Lavender & Vanilla Handmade Luxury Soap,275.00,Beauty & Hygiene,4.996872,3.941125
7,907,Sauce - Aglio Olio,290.00,Gourmet & World Food,4.996029,4.268137
8,25,Veggie Cutter,195.00,"Kitchen, Garden & Pets",4.995369,3.732320
9,734,Lavender Try Me Personal Care Gift Set,850.00,Beauty & Hygiene,4.994356,4.051660


## Make the hybrid recommender system

In [ ]:
# Hybrid Recommender System
def hybrid_recommend(user_id, budget,dataset):

    # Calculate the average rating for collaborative recommendations
    collaborative_avg_rating = np.mean(dataset['clf_ratings'])

    # Calculate the average rating for content-based recommendations
    content_based_avg_rating = np.mean(dataset['cbf_ratings'])

    # Calculate the final recommendations based on weighted average ratings
    dataset['weighted_avg'] = (dataset['clf_ratings'] * collaborative_avg_rating + dataset['cbf_ratings']* content_based_avg_rating) / 2

    hybrid_recommendations = dataset[['product_id','weighted_avg','discount_price']].values.tolist()

    # Sort the recommendations by rating in descending order
    hybrid_recommendations.sort(key=lambda x: x[1], reverse=True)

    # Filter the recommendations based on the user's budget
    filtered_recommendations = []
    total_cost = 0
    for product, rating,price in hybrid_recommendations:
        product_cost = price
        if total_cost + product_cost <= budget:
            filtered_recommendations.append(product)
            total_cost += product_cost
        if len(filtered_recommendations) >= 10:
            break

    return filtered_recommendations

# Example usage
user_id = 126093
budget = 10000
recommendations = hybrid_recommend(user_id, budget,combined)
print(recommendations)

[920.0, 276.0, 813.0, 45.0, 669.0, 18.0, 907.0, 698.0, 778.0, 734.0]


# Model Deployment 

## save and load cbfmodel as .h5 file


In [ ]:
#download pickle (h5) file
model = cbfmodel

model.save('cbf_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
loaded_cbf_model = load_model('cbf_model.h5')

## save and load clfmodel as .h5 file

In [ ]:
#download pickle (h5) file
model = clfmodel

model.save('clf_model.h5')

In [ ]:
# Load the model from the .h5 file
loaded_clf_model = load_model('clf_model.h5')

## deploy model

In [ ]:
#load the dataset
df = pd.read_csv('/content/drive/MyDrive/final_dataset.csv')
df = df.drop(["Unnamed: 0"],axis=1)

dataset = df.copy()
scaler = MinMaxScaler()
dataset[["actual_price", "discount_price", "ratings", "add_to_cart_order"]] = scaler.fit_transform(dataset[["actual_price", "discount_price", "ratings", "add_to_cart_order"]])
dataset['category_id'] = LabelEncoder().fit_transform(dataset['category'])

# get recommendation from collaborative filtering
def collaborative_filtering_model(df,dataset, user_id,clfmodel):
  # Generate item recommendations for the specific user
  num_users = len(dataset['user_id'].unique())
  num_items = len(dataset['product_id'].unique())
  user_items = np.full((num_items,), user_mapping[user_id])
  item_ids = np.arange(num_items)

  # Predict ratings for the user-item combinations
  predictions = clfmodel.predict([user_items, item_ids]).flatten()

  # Create a DataFrame of predictions for the user
  recommendations_df = pd.DataFrame({'product_id': item_mapping.keys(), 'rating': predictions})

  # Sort the DataFrame by predicted rating in descending order
  recommendations_df = recommendations_df.sort_values('rating', ascending=False)

  # Select the top 10 recommendations
  top_10_recs = recommendations_df.nlargest(10, 'rating')
  clfrecs = top_10_recs[['product_id', 'rating']]
  clfrecs = clfrecs.rename(columns={'rating': 'clf_ratings'})
  clfinfo = pd.merge(clfrecs,df,on='product_id')
  clfinfo = clfinfo.drop_duplicates(subset='product_id', keep='first')
  clfinfo = clfinfo.drop(['user_id','add_to_cart_order','reordered','actual_price','ratings','brand'],axis=1) 
  # Identify the index of the column to move
  column_index = clfinfo.columns.get_loc('clf_ratings')

  # Create a list of column names in the desired order
  new_columns = list(clfinfo.columns[:column_index]) + list(clfinfo.columns[column_index+1:]) + ['clf_ratings']

  # Reorder the columns in the DataFrame
  clfinfo = clfinfo[new_columns]


  return clfinfo

# get recommendations from content based ratings
def content_based_model(df, dataset, user_id, cbfmodel):
  # Predict the ratings for all products in the dataset
  all_products_data = dataset[dataset['user_id'] != user_id]
  X_all = [all_products_data['category_id'], all_products_data[["discount_price", "reordered", "add_to_cart_order"]]]
  predicted_ratings = cbfmodel.predict(X_all).flatten()


  # Create a list of tuples containing product ID and predicted rating
  product_ratings = list(zip(all_products_data['product_id'], predicted_ratings))

  # Sort the product ratings by predicted rating in descending order
  product_ratings.sort(key=lambda x: x[1], reverse=True)

  # Get the top 10 recommendations without duplicate product IDs
  recommended_products = []
  recommended_product_ids = set()

  for product_id, _ in product_ratings:
      if product_id not in recommended_product_ids:
          recommended_products.append((product_id, _))
          recommended_product_ids.add(product_id)
      if len(recommended_products) >= 10:
          break

  # Filter the original dataset based on the recommended product IDs
  top_10_recommendations = all_products_data[all_products_data['product_id'].isin([p[0] for p in recommended_products])]

  # Remove duplicate product IDs from the recommendations
  top_10_recommendations = top_10_recommendations.drop_duplicates(subset='product_id')

  # Keep the top 10 recommendations (if there are more than 10 after removing duplicates)
  top_10_recommendations = top_10_recommendations.head(10)

  pred_rating = pd.DataFrame(product_ratings)
  pred_rating.columns =['product_id','cbf_ratings']
  pred_rating = pred_rating.drop_duplicates(subset='product_id', keep='first')
  pred_rating['cbf_ratings'] = pred_rating['cbf_ratings'] * (df['ratings'].max() - df['ratings'].min() ) + df['ratings'].min()
  pred_rating['cbf_ratings'] = pred_rating['cbf_ratings'].clip(lower=0.0, upper=5.0)
  cbfrecs = top_10_recommendations[['product_id', 'name','discount_price','category']]
  cbfinfo = pd.merge(cbfrecs, pred_rating, on='product_id')
  cbfinfo = cbfinfo.drop_duplicates(subset='product_id', keep='first')
  cbfinfo['discount_price'] = cbfinfo['discount_price'] * (df['discount_price'].max()- df['discount_price'].min() ) + df['discount_price'].min()

  return cbfinfo , pred_rating


# recommendingproducts id
def finaldataset(df,dataset,user_id,cbfmodel,clfmodel):
  # get final collaborative filtering top 10 result data
  clfinfo = collaborative_filtering_model(df,dataset, user_id,clfmodel)
  cbfinfo , pred_rating = content_based_model(df, dataset, user_id, cbfmodel)
  finalclf =  pd.merge(clfinfo, pred_rating, on='product_id', how='left')

  # get final content-based filtering top 10 result data
  product_ids = [x for x in cbfinfo['product_id']] 
  user_ids = np.full(len(product_ids), user_mapping[user_id])
  item_ids = np.array([item_mapping[product_id] for product_id in product_ids])
  predictions = clfmodel.predict([user_ids, item_ids]).flatten()
  user_product_ratings_df = pd.DataFrame({'product_id': product_ids,'clf_ratings': predictions})
  finalcbf =  pd.merge(cbfinfo, user_product_ratings_df, on='product_id', how='left')
  
  # combine the result
  combined = pd.concat([finalclf, finalcbf], axis=0)
  combined['cbf_ratings'] = combined['cbf_ratings'] * (df['ratings'].max() - df['ratings'].min() ) + df['ratings'].min()
  combined['cbf_ratings'] = combined['cbf_ratings'].clip(lower=0.0, upper=5.0)
  combined['clf_ratings'] = combined['clf_ratings'] * (df['ratings'].max() - df['ratings'].min() ) + df['ratings'].min()
  combined['clf_ratings'] = combined['clf_ratings'].clip(lower=0.0, upper=5.0)
  combined = combined.drop_duplicates(subset='product_id', keep='first')

  return combined

def hybrid_recommend(df,dataset,user_id,cbfmodel,clfmodel,budget):
    all_recs = finaldataset(df,dataset,user_id,cbfmodel,clfmodel)

    # Calculate the average predicted rating for both recommendations
    collaborative_avg_rating = np.mean(all_recs['clf_ratings'])
    content_based_avg_rating = np.mean(all_recs['cbf_ratings'])

    # Calculate the final recommendations based on weighted average ratings

    all_recs['weighted_avg'] = (all_recs['clf_ratings'] * collaborative_avg_rating + all_recs['cbf_ratings']* content_based_avg_rating) / 2
    hybrid_recommendations = all_recs[['product_id','weighted_avg','discount_price']].values.tolist()

    # Sort the recommendations by rating in descending order
    hybrid_recommendations.sort(key=lambda x: x[1], reverse=True)

    # Filter the recommendations based on the user's budget
    filtered_recommendations = []
    total_cost = 0
    for product, rating,price in hybrid_recommendations:
        product_cost = price
        if total_cost + product_cost <= budget:
            filtered_recommendations.append(product)
            total_cost += product_cost
        if len(filtered_recommendations) >= 10:
            break

    return filtered_recommendations, total_cost



In [ ]:
# Example usage
user_id = 126093
budget = 10000
recommendations, total_cost = hybrid_recommend(df,dataset,user_id,loaded_cbf_model,loaded_clf_model,budget)
print(recommendations)
print(total_cost)

1/1 [==============================] - 0s 19ms/step
[925.0, 94.0, 977.0, 45.0, 920.0, 669.0, 169.0, 907.0, 25.0, 734.0]
4617.95
